In [ ]:
pip install pandas



SyntaxError: invalid syntax (<ipython-input-1-376be87a97f7>, line 1)

In [ ]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.9 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.4 MB/s eta 0:00:00


In [ ]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from bs4 import BeautifulSoup
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [ ]:
os.environ["OPENAI_API_KEY"] = "Enter Key here"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def extract_htmlcode(path):
  with open(path,"r", encoding="utf-8") as file:
    soup =BeautifulSoup(file,"html.parser")
    return soup.get_text()

In [ ]:

document=extract_htmlcode('/content/drive/My Drive/GenAI10K/10KAnalysis.html')

In [ ]:
Embeddings = OpenAIEmbeddings()

<ipython-input-10-a5bbd220731c>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  Embeddings = OpenAIEmbeddings()


In [ ]:
vector_store = FAISS.from_texts([document], Embeddings)


In [ ]:
retriever=vector_store.as_retriever()

In [ ]:
llm=ChatOpenAI( model_name="gpt-3.5-turbo",temperature=0)

<ipython-input-13-f9ae4ebbc334>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI( model_name="gpt-3.5-turbo",temperature=0)


In [ ]:
# this code is using conversation buffer which will just store the previous queationa and pass it LLM when a new question is asked it will append the chat history and send it to LLM.
#disadvantage is If the chat history grows too long, the context may exceed token limits.
# Alternatively u can use create_history_aware_retriever  : This it will Reframes the current query by incorporating chat history before retrieving relevant documents.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, memory=memory
)
query = "what is the revenue of Apple in 2022."
response = rag_chain.run(query)
print(response)

<ipython-input-14-93b4a4187694>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
<ipython-input-14-93b4a4187694>:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)


The revenue of Apple in 2022 was $394.3 billion.


In [ ]:
#Try the same with create_history_aware_retriever
#Provides instructions to the model on how to rephrase the query based on past interactions.
#contextualize_q_system_prompt is a system message
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
#("human", "{input}") ->Represents the current user query that needs to be rephrased.
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
system_prompt = (
    "You are an AI assistant specializing in financial insights. "
    "Explain financial concepts simply and concisely. "
    "When relevant, describe how a chart or graph could help visualize trends "
    "or comparisons (e.g., a line graph for revenue trends, a bar chart for assets vs. liabilities). "
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
store = {}

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
def chatbot(input_text, session_id="default"):
    # Run the input through the RAG pipeline
    response = conversational_rag_chain.invoke(
    {"input":input_text },
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)


    answer=response["answer"]
    follow_up_suggestions = [
        "Would you like to compare this year's data with previous years?",
        "Do you want insights on a specific company's financial health?",
        "Would you like a breakdown of key financial metrics?"
    ]

    # Format response with a follow-up question
    final_response = f"{answer}\n\n💡 {follow_up_suggestions[0]}"
    return final_response


In [ ]:
print(chatbot("How did Tesla’s revenue change over the last three years?", session_id="user1"))



Tesla's revenue changed significantly over the last three years. In 2021, Tesla's revenue represented 100% of the total revenue, but in 2022, it decreased to 12.08% and further dropped to 13.99% in 2023. This indicates a substantial decrease in revenue contribution from Tesla over the three-year period.

💡 Would you like to compare this year's data with previous years?
